In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspellchecker spacy -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 13.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
import nltk
import spacy
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from tabulate import tabulate
from nltk.corpus import wordnet as wn
from spellchecker import SpellChecker
from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

# Load the Dataset

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/SideBoys/sem 7/AIHC/mtsamples.csv')

In [ ]:
df.head()

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


# Data Preparation

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4999 entries, 0 to 4998
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         4999 non-null   int64 
 1   description        4999 non-null   object
 2   medical_specialty  4999 non-null   object
 3   sample_name        4999 non-null   object
 4   transcription      4966 non-null   object
 5   keywords           3931 non-null   object
dtypes: int64(1), object(5)
memory usage: 234.5+ KB


In [ ]:
df = df.dropna(subset=['transcription'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4966 entries, 0 to 4998
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         4966 non-null   int64 
 1   description        4966 non-null   object
 2   medical_specialty  4966 non-null   object
 3   sample_name        4966 non-null   object
 4   transcription      4966 non-null   object
 5   keywords           3898 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.6+ KB


# Data Cleaning

In [ ]:
def remove_special_characters(text):
    remove_special = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return ''.join(remove_special)

In [ ]:

for col in df:
  if col!= 'Unnamed: 0' and col!= 'keywords':
    df[col] = df[col].apply(remove_special_characters)

<ipython-input-10-98d0c46df82e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(remove_special_characters)


In [ ]:
df.head()

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23yearold white female presents with compla...,Allergy Immunology,Allergic Rhinitis,SUBJECTIVE This 23yearold white female presen...,"allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass,Bariatrics,Laparoscopic Gastric Bypass Consult 2,PAST MEDICAL HISTORY He has difficulty climbin...,"bariatrics, laparoscopic gastric bypass, weigh..."
2,2,Consult for laparoscopic gastric bypass,Bariatrics,Laparoscopic Gastric Bypass Consult 1,HISTORY OF PRESENT ILLNESS I have seen ABC to...,"bariatrics, laparoscopic gastric bypass, heart..."
3,3,2D MMode Doppler,Cardiovascular Pulmonary,2D Echocardiogram 1,2D MMODE 1 Left atrial enlargement with left...,"cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,4,2D Echocardiogram,Cardiovascular Pulmonary,2D Echocardiogram 2,1 The left ventricular cavity size and wall t...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


# Text Preprocessing

In [ ]:
spell = SpellChecker()
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def text_preprocessing(text):
    # Tokenization
    words = word_tokenize(text)

    # Lowercasing and removing punctuation
    words = [re.sub(r'[^\w\s]', '', word.lower()) for word in words]

    # Stop word removal
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Stemming
    words = [lemmatizer.lemmatize(word) for word in words]

    return ' '.join(words)

In [ ]:
for col in df:
  if col!= 'Unnamed: 0' and col!= 'keywords':
    df[col] = df[col].apply(text_preprocessing)

In [ ]:
df.head()

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,23yearold white female present complaint allergy,allergy immunology,allergic rhinitis,subjective 23yearold white female present comp...,"allergy / immunology, allergic rhinitis, aller..."
1,1,consult laparoscopic gastric bypass,bariatrics,laparoscopic gastric bypass consult 2,past medical history difficulty climbing stair...,"bariatrics, laparoscopic gastric bypass, weigh..."
2,2,consult laparoscopic gastric bypass,bariatrics,laparoscopic gastric bypass consult 1,history present illness seen abc today pleasan...,"bariatrics, laparoscopic gastric bypass, heart..."
3,3,2d mmode doppler,cardiovascular pulmonary,2d echocardiogram 1,2d mmode 1 left atrial enlargement left atrial...,"cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,4,2d echocardiogram,cardiovascular pulmonary,2d echocardiogram 2,1 left ventricular cavity size wall thickness ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


# NER

In [ ]:
!pip install -U spacy
!pip install scispacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 17.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.9 MB/s eta 0:00:00
  Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
  Created wheel for nmslib: filename=nmslib-2.1.1-cp310-cp310-linux_x86_64.whl size=13578642 sha256=3ecfcccf7a67f48990444436ad9e2f81357a2215934d1162db0f64d32831e745
  Stored in directory: /root/.cache/pip/wheels/21/1a/5d/4cc754a5b1a88405cad184b76f823897a63a8d19afcd4b9314
Successfully built nmslib
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.2
    Uninstalling wasabi-1.1.2:
      Successfully uninstalled wasabi-1.1.2
  Attempting uninstall: typer
    Found existing installation: typer 0.9.0
    Uninstalling typer-0.9.0:
      Successfully unins

In [ ]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_ner_bc5cdr_md-0.5.1.tar.gz -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 MB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install render -q

  Preparing metadata (setup.py) ... done


In [ ]:
import scispacy
import spacy
import en_ner_bc5cdr_md
from spacy import displacy
import pandas as pd

/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_ner_bc5cdr_md' (0.5.1) was trained with spaCy v3.4.1 and may not be 100% compatible with the current version (3.6.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  


In [ ]:
nlp_bc = en_ner_bc5cdr_md.load()
doc = nlp_bc(df['transcription'][2])
displacy_image = displacy.render(doc, jupyter=True,style='ent', options={"colors": {'DISEASE ': "#85C1E9", "CHEMICAL ": "#ff6961"}})

In [ ]:
doc = nlp_bc(df['transcription'][4])
displacy_image = displacy.render(doc, jupyter=True,style='ent')

In [ ]:
doc = nlp_bc(df['transcription'][26])
displacy_image = displacy.render(doc, jupyter=True,style='ent')

In [ ]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

prostate symptom bladder neck obstruction 175 216 DISEASE
creatinine 322 332 CHEMICAL
granulomatous disease 490 511 DISEASE
cysto tur prostate 564 582 DISEASE
bactrim 730 737 CHEMICAL
prostate benign bladder neck obstruction 834 874 DISEASE


In [ ]:
from spacy.gold import GoldParse
from spacy.scorer import Scorer

def evaluate(ner_model, examples):
    scorer = Scorer()
    for input_, annot in examples:
        doc_gold_text = ner_model.make_doc(input_)
        gold = GoldParse(doc_gold_text, entities=annot)
        pred_value = ner_model(input_)
        scorer.score(pred_value, gold)
    return scorer.scores

results = evaluate(ner_model, examples)